In [ ]:
from pandas import Series, DataFrame
import pandas as pd
import re

df = pd.read_csv('NYC_Restaurants.csv',dtype=unicode)

Q1 Create a unique name for each restaurant

Approach: added a new column that concatenated the DBA, BUILDING, STREET,
Columns with spaces in between

In [ ]:
df['RESTAURANT']=df['DBA']+" "+df['BUILDING']+" "+df['STREET']+" "+df["ZIPCODE"]
print df['RESTAURANT'][:10]

Q2 How many restaurants are included in the data?

Approach: created a unique restaurants dataframe and deleted all duplicates\
in the RESTAURANT column
used the len function to find the number of records in the uniqueResturants\
data frame

In [ ]:
uniqueRestaurants = df.drop_duplicates(subset='RESTAURANT')
print "Number of restaurants:", len(uniqueRestaurants)

Q3 How many chains are there?

Approach: created a data frame with a value count of the 'DBA' column
used the unique restaurants dataframe so that repeat offenders wouldn't be
counted twice
created a mask for when the value count was greater than or equal to 2
applied the mask to the list of unique restaurants
found the length (len) of the new dataframe

In [ ]:
#create dataframe with the count of each restaurant of a chain
numberOfEach = uniqueRestaurants['DBA'].value_counts()
#create a mask to find resturants with 2 or more locations (chains)
chainMask = (numberOfEach >= 2)
chains = numberOfEach[chainMask]

print "Number of chains:", len(chains)


Q4 Plot a bar graph of the top 20 most popular chains?

Approach: take the chains data frame and create a bar plot of the 
top 20 chains, which would be the ones with highest value counts
import %pylab inline to show bar plot

In [ ]:
%pylab inline
top20_chains = chains[:20]
top20_chains.plot(kind='bar')
print sum(chains)

Q5 What fraction of all restaurants are chains?

Approach: we need the total number of restaurants and the total 
number of restaurant in chains  restaurants in chains / 
total number of restaurants

In [ ]:
percentage_of_chains = float(sum(chains))/float(len(uniqueRestaurants))
print 'Fraction of restaurants that are chains '+ "%.2f" % float(percentage_of_chains *100) + "%"

Q6 Plot all non-chain restaurants in each borrough

Approach: First we created a series that included only borroughs and unique restaurants,we derived this series from the unique restaurant dataframe from problem 2. Then I filtered out all the missing borrough values by dropping them. I then created a mask and applied it to the dataframe that we removed the missing values from. So at this point only non_chain restaurants should show. Once applied, the dataframe became a series, so i converted it back to a series so that it could be merged. Then i used value counts on the boroughs and plotted it 

In [ ]:
#building series of only borroughs and restaurants
rest_borrough = uniqueRestaurants[['DBA','BORO']]

#non chain mask
non_chainMask = (numberOfEach< 2)
non_chain = numberOfEach[non_chainMask]

non_chain_df = DataFrame(non_chain.index) #this creates a dataframe from the series that 
                                          #we applied the mask to, that way we can merge them
non_chain_df.columns = ['Restaurants']#name a column in the new dataframe


#here I merge the datframes and apply the value_counts to it
merged_df = pd.merge(non_chain_df, rest_borrough,left_on='Restaurants' , right_on='DBA', suffixes =['_non_chain','_rest'])
merged_df_nulls = merged_df.replace("Missing",NaN).dropna(how = "any")
answerQ7 = DataFrame(merged_df_nulls["BORO"].value_counts())
answerQ7.plot(kind='bar')

Q7

Approach: We used the uniqueRestaurants DF and removed all the missing info. We them used value counts and this gave us the number of restaurants in each borrough. I then converted that value_counts output into a DF so that I could combine it with other info. THen I added the number of nonchain restaurants to this df, i used the values from Q7. I then divided to receive a percentage and then plotted that.

In [ ]:
#create a df that counts all restaurants per borrough and elimnates missing info\n",
pre_lim = uniqueRestaurants[['BORO','DBA']].replace("Missing",NaN).dropna(how = "any")
all_rest_boro = pre_lim['BORO'].value_counts() #this is shows the total number of restaurants in each borrough"
all_rest_boro_df = DataFrame(all_rest_boro)
all_rest_boro_df.columns = ['Total in Borrough']
all_rest_boro_df['NonChain in Borrough'] = answerQ7["BORO"]
all_rest_boro_df['Percent'] = all_rest_boro_df['NonChain in Borrough']/all_rest_boro_df['Total in Borrough']
all_rest_boro_df['Percent'].plot(kind = "bar")

Q8 Plot the popularity of cuisines

Approach: Collect all cuisine descriptions, see the frequency of these cuisines by collecting the top 20, then make histogram of the top 20 cuisines 

In [ ]:
#get the cuisine types 
df['CUISINE DESCRIPTION']

#see frequency of cuisines 
vc = df['CUISINE DESCRIPTION'].value_counts()
vc[:20]

#plot histogram of top 20 cuisines 
top_20_vc = vc[:20]
top_20_vc.plot(kind='bar')

In [ ]:
Q9

Approach 

In [ ]:
#make dataframe with resturant and cuisine description

restaurant_cuisine_df = df[['RESTAURANT', 'CUISINE DESCRIPTION']]

#print restaurant_cuisine_df


#find restaurants that are violators
#mask_restaurant_cuisine_nonviolators = df['VIOLATION CODE'].isnull()

#print mask_restaurant_cuisine_nonviolators

#combine mask with datafram
#mask = (restaurant_cuisine_df & mask_restaurant_cuisine_nonviolators)

mask = df['VIOLATION CODE'].isnull()

#apply the mask
mask_nonviolators = df[mask]

count_nonviolators = mask_nonviolators['DBA'].value_counts()

print(count_nonviolators[:20])

#plot histogram of top 20 cuisines 


Q10: What Cuisines tend to be the cleanest?

Approach

Q11 What are the most common violations in each borough?

APPROACH: I first assigned a dummy column to each inspection for counting purpose.
Then I cleared out the data by substituting all missing BORO with NaN. Finally use pivot table to organize 
#the number of each type of violations by borough and listed the top violation with the function .idxmax().

In [ ]:
df['count'] = 1 # dumpy value
    


vioBoro = df[['VIOLATION DESCRIPTION','BORO','count']]

#most = vioBoro['VIOLATION DESCRIPTION'].value_counts()
#print most[:5]

mask_missing = (vioBoro['BORO'] == 'Missing')
vioBoro.ix[mask_missing, 'BORO'] = np.nan # This sets the value to NaN

violations = pd.pivot_table(vioBoro,
                         index='VIOLATION DESCRIPTION',
                         columns='BORO',
                         values='count',
                         aggfunc=sum, #what do I do with the data if there is a collision
                         fill_value=0) #fill non-existed violation with 0.
fig, (ax1, ax2, ax3, ax4, ax5) = subplots(nrows=5, ncols=1, sharex=True,sharey=True, figsize\
                                    =(8,12))
violations['BRONX'].plot(ax=ax1)
violations['BROOKLYN'].plot(ax=ax2)
violations['MANHATTAN'].plot(ax=ax3)
violations['QUEENS'].plot(ax=ax4)
violations['STATEN ISLAND'].plot(ax=ax5)

topVio = violations.idxmax()


print topVio

In [ ]:
#Q12 What are the most common violations per borough, 
#after normalizing for the relative abundance of each violation?
# Step 1: Get overall frequencies
frequency = df['VIOLATION CODE'].dropna().value_counts()

#Step 2: Create a new column that records total frquency

totalFreq = DataFrame(frequency[df['VIOL CODE'].dropna()])
print totalFreq

df['normalize'] = df['count']/totalFreq["VIOL CODE"]
   
#Create a pivot table with the percentage column and the Boro column
norm_vioBoro = df[['VIOLATION DESCRIPTION','BORO','normalize']]
norm_violations = pd.pivot_table(norm_vioBoro,
                         index='VIOLATION DESCRIPTION',
                         columns='BORO',
                         values='normalize',
                         aggfunc=sum, #what do I do with the data if there is a collision
                         fill_value=0) #fill non-existed violation with 0.
topVio = norm_violations.idxmax()


